In [15]:
import matplotlib

%matplotlib inline

In [85]:
# The first thing to do is to import the relevant packages
# that I will need for my script, 
# these include the Numpy (for maths and arrays)
# and csv for reading and writing csv files
# If i want to use something from this I need to call 
# csv.[function] or np.[function] first

import csv as csv 
import numpy as np

# Open up the csv file in to a Python object
data_all = []
with open('events.csv', encoding="utf8") as train_file:
    csv_reader = csv.reader(train_file, delimiter=';', quotechar='"')
    for row in csv_reader:
        data_all.append(row)
data_all = np.array(data_all)
data = data_all[1::]


In [199]:
import pandas as pd
import re

data = pd.read_csv('events.csv', sep=';')

#population_age = df.Link.astype(np.float)
#print(data.Description)
#print(data.Details)

df = data[['Description','Details']]

df[0:10]

,Description,Details
0,"Rencontre autour des nouveaux usages, de la pa...",Que recouvre aujourd’hui la notion de particip...
1,Découverte des risques toxicologiques des cham...,Découvrez les risques toxicologiques par des d...
2,Une enquête sur un empoisonnement,Devenez détectives et élucidez l'enquête qui v...
3,Apprendre à dépister des substances toxiques,"En laboratoire, comme dans la vie de tous les ..."
4,Expositions et animations autour de l'expérime...,**Le Réseau des maisons de l’architecture orga...
5,Agence de marketing d'influence,NaN
6,dans le cadre du festival Bar-Bars,Déroutant et imprévisible : Gratuit ne fait ja...
7,dans le cadre du festival Bar-Bars,Dj Barbarpapa en exclusivité au Barablabla le ...
8,dans le cadre du festival Bar-Bars,Nouch'Ma est un duo guitare/violon naviguant e...
9,dans le cadre du festival Bar-Bars,"Bas d'Plafond, c'est quelque part entre Babylo..."


## Netoyage des données

LDA a besoin que chaque phrase soit découpé sous forme d'un tableau de mots minuscules. On réfléchira par la suite sur le fait que plusieurs

In [179]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')


for item in data.Description:
    if type(item) is str :
        raw = item.lower()
        token = tokenizer.tokenize(raw)
        #print(token)




In [204]:
#ON SUPPRIME LES STOP WORDS (mot du langage)
from stop_words import get_stop_words

# create English stop words list
#en_stop = get_stop_words('en')

# create French stop words list
fr_stop = get_stop_words('fr')

print(fr_stop[20:60])


['auront', 'aussi', 'autre', 'aux', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', 'avons', 'ayant', 'ayez', 'ayons', 'bon', 'car', 'ce', 'ceci', 'cela', 'ces', 'cet', 'cette', 'ceux', 'chaque', 'ci', 'comme', 'comment', 'd', 'dans', 'de', 'dedans', 'dehors', 'depuis', 'des', 'deux', 'devoir', 'devrait']


# Stop words
Certain parts of English speech, like conjunctions (“for”, “or”) or the word “the” are meaningless to a topic model. 
These terms are called stop words and need to be removed from our token list.

#### On enlève les STOP WORDS des phrases

In [216]:

tokens_stop_serie = pd.Series()
i=0

for item in data.Description:
        raw = str(item) +' '+ str(data.Details[i])
        token = tokenizer.tokenize(raw.lower())
        stopped_tokens = [i for i in token if not i in fr_stop]
        tokens_stop_serie = tokens_stop_serie.set_value(i,stopped_tokens)
        i+=1
        
print(tokens_stop_serie[0:8])
    


0    [rencontre, autour, usages, participation, nou...
1    [découverte, risques, toxicologiques, champign...
2    [enquête, empoisonnement, devenez, détectives,...
3    [apprendre, dépister, substances, toxiques, la...
4    [expositions, animations, autour, expérimentat...
5                  [agence, marketing, influence, nan]
6    [cadre, festival, bar, bars, déroutant, imprév...
7    [cadre, festival, bar, bars, dj, barbarpapa, e...
dtype: object


# Stemming
Stemming words is another common NLP technique to reduce topically similar words to their root. 
For example, “nantaise,” “nantes,” “nantais,” all have similar meanings; stemming reduces those terms to “nant” 
This is important for topic modeling, which would otherwise view those terms as separate entities and reduce 
their importance in the model

In [217]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import FrenchStemmer

stemmer = FrenchStemmer()
tokens_stem_serie = pd.Series()
i = 0
# Create p_stemmer of class PorterStemmer
# p_stemmer = PorterStemmer()

for stop_word_token in tokens_stop_serie:
    # stem token
    stemmed_tokens = [stemmer.stem(i) for i in stop_word_token]
    tokens_stem_serie = tokens_stem_serie.set_value(i,stemmed_tokens)
    i += 1

print(tokens_stem_serie[0:8])


0    [rencontr, autour, usag, particip, nouvel, for...
1    [découvert, risqu, toxicolog, champignon, frai...
2    [enquêt, empoison, deven, détect, élucid, enqu...
3    [apprendr, dépist, substanc, toxiqu, laboratoi...
4    [exposit, anim, autour, expériment, class, cm2...
5                    [agenc, marketing, influenc, nan]
6    [cadr, festival, bar, bar, dérout, imprévisibl...
7    [cadr, festival, bar, bar, dj, barbarpap, excl...
dtype: object


# Constructing a document-term matrix

In [223]:
from gensim import corpora, models

dictionary = corpora.Dictionary(tokens_stem_serie)

corpus = [dictionary.doc2bow(token) for token in tokens_stem_serie]

# The tuples are (term ID, term frequency)
print(corpus[0])


ldamodel = models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)



INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(8055 unique tokens: ['tabli', 'aftershow', 'silhouet', 'rideau', 'sj53clqrdf8']...) from 950 documents (total 43296 corpus positions)
INFO:gensim.models.ldamodel:using symmetric alpha at 0.1
INFO:gensim.models.ldamodel:using symmetric eta at 0.00012414649286157667
INFO:gensim.models.ldamodel:using serial LDA version on this node


[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 5), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 3), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1)]


INFO:gensim.models.ldamodel:running online LDA training, 10 topics, 20 passes over the supplied corpus of 950 documents, updating model once every 950 documents, evaluating perplexity every 950 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:-11.745 per-word bound, 3432.4 perplexity estimate based on a held-out corpus of 950 documents with 43296 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #950/950
INFO:gensim.models.ldamodel:topic #0 (0.100): 0.012*"nan" + 0.008*"http" + 0.008*"www" + 0.006*"com" + 0.005*"facebook" + 0.004*"s" + 0.004*"nant" + 0.004*"bar" + 0.004*"ven" + 0.004*"découvr"
INFO:gensim.models.ldamodel:topic #3 (0.100): 0.012*"http" + 0.009*"www" + 0.008*"com" + 0.007*"nant" + 0.006*"facebook" + 0.005*"visit" + 0.005*"exposit" + 0.004*"s" + 0.004*"bar" + 0.004*"présent"
INFO:gensim.models.ldamodel:topic #6 (0.100): 0.005*"utilis" + 0.005*"nant" + 0.005*"nan" + 0.004*"mis" + 0.004*"découvert" + 0.004*"organis"

In [224]:
print(ldamodel.print_topics(num_topics=10, num_words=2))

INFO:gensim.models.ldamodel:topic #0 (0.100): 0.021*"nan" + 0.008*"nant"
INFO:gensim.models.ldamodel:topic #1 (0.100): 0.016*"contenu" + 0.015*"utilis"
INFO:gensim.models.ldamodel:topic #2 (0.100): 0.013*"nant" + 0.007*"projet"
INFO:gensim.models.ldamodel:topic #3 (0.100): 0.009*"nant" + 0.008*"visit"
INFO:gensim.models.ldamodel:topic #4 (0.100): 0.012*"nant" + 0.006*"amp"
INFO:gensim.models.ldamodel:topic #5 (0.100): 0.014*"http" + 0.012*"www"
INFO:gensim.models.ldamodel:topic #6 (0.100): 0.012*"imag" + 0.009*"text"
INFO:gensim.models.ldamodel:topic #7 (0.100): 0.015*"bar" + 0.014*"http"
INFO:gensim.models.ldamodel:topic #8 (0.100): 0.056*"bar" + 0.052*"http"
INFO:gensim.models.ldamodel:topic #9 (0.100): 0.010*"nant" + 0.007*"e"


[(0, '0.021*"nan" + 0.008*"nant"'), (1, '0.016*"contenu" + 0.015*"utilis"'), (2, '0.013*"nant" + 0.007*"projet"'), (3, '0.009*"nant" + 0.008*"visit"'), (4, '0.012*"nant" + 0.006*"amp"'), (5, '0.014*"http" + 0.012*"www"'), (6, '0.012*"imag" + 0.009*"text"'), (7, '0.015*"bar" + 0.014*"http"'), (8, '0.056*"bar" + 0.052*"http"'), (9, '0.010*"nant" + 0.007*"e"')]


In [80]:
import numpy as np
import lda
import lda.datasets

X = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()
X.shape
(395, 4258)
X.sum()
84010
model = lda.LDA(n_topics=20, n_iter=1500, random_state=1)
model.fit(X)  # model.fit_transform(X) is also available
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]     
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))
    



INFO:lda:n_documents: 395
INFO:lda:vocab_size: 4258
INFO:lda:n_words: 84010
INFO:lda:n_topics: 20
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -1051748
INFO:lda:<10> log likelihood: -719800
INFO:lda:<20> log likelihood: -699115
INFO:lda:<30> log likelihood: -689370
INFO:lda:<40> log likelihood: -684918
INFO:lda:<50> log likelihood: -681322
INFO:lda:<60> log likelihood: -678979
INFO:lda:<70> log likelihood: -676598
INFO:lda:<80> log likelihood: -675383
INFO:lda:<90> log likelihood: -673316
INFO:lda:<100> log likelihood: -672761
INFO:lda:<110> log likelihood: -671320
INFO:lda:<120> log likelihood: -669744
INFO:lda:<130> log likelihood: -669292
INFO:lda:<140> log likelihood: -667940
INFO:lda:<150> log likelihood: -668038
INFO:lda:<160> log likelihood: -667429
INFO:lda:<170> log likelihood: -666475
INFO:lda:<180> log likelihood: -665562
INFO:lda:<190> log likelihood: -664920
INFO:lda:<200> log likelihood: -664979
INFO:lda:<210> log likelihood: -664722
INFO:lda:<220> log likelihood: -

Topic 0: british churchill sale million major letters west britain
Topic 1: church government political country state people party against
Topic 2: elvis king fans presley life concert young death
Topic 3: yeltsin russian russia president kremlin moscow michael operation
Topic 4: pope vatican paul john surgery hospital pontiff rome
Topic 5: family funeral police miami versace cunanan city service
Topic 6: simpson former years court president wife south church
Topic 7: order mother successor election nuns church nirmala head
Topic 8: charles prince diana royal king queen parker bowles
Topic 9: film french france against bardot paris poster animal
Topic 10: germany german war nazi letter christian book jews
Topic 11: east peace prize award timor quebec belo leader
Topic 12: n't life show told very love television father
Topic 13: years year time last church world people say
Topic 14: mother teresa heart calcutta charity nun hospital missionaries
Topic 15: city salonika capital buddhist c

In [94]:
doc_topic = model.doc_topic_
for i in range(10):
    print("{} (top topic: {})".format(titles[i], doc_topic[i].argmax()))

0 UK: Prince Charles spearheads British royal revolution. LONDON 1996-08-20 (top topic: 8)
1 GERMANY: Historic Dresden church rising from WW2 ashes. DRESDEN, Germany 1996-08-21 (top topic: 13)
2 INDIA: Mother Teresa's condition said still unstable. CALCUTTA 1996-08-23 (top topic: 14)
3 UK: Palace warns British weekly over Charles pictures. LONDON 1996-08-25 (top topic: 8)
4 INDIA: Mother Teresa, slightly stronger, blesses nuns. CALCUTTA 1996-08-25 (top topic: 14)
5 INDIA: Mother Teresa's condition unchanged, thousands pray. CALCUTTA 1996-08-25 (top topic: 14)
6 INDIA: Mother Teresa shows signs of strength, blesses nuns. CALCUTTA 1996-08-26 (top topic: 14)
7 INDIA: Mother Teresa's condition improves, many pray. CALCUTTA, India 1996-08-25 (top topic: 14)
8 INDIA: Mother Teresa improves, nuns pray for "miracle". CALCUTTA 1996-08-26 (top topic: 14)
9 UK: Charles under fire over prospect of Queen Camilla. LONDON 1996-08-26 (top topic: 8)
